# Rigid Transformations

This *Jupyter Notebook* is dedicated to the study of **Rigid Transformations** and its properties. The content is guided by the [Foundations of Robotics]((https://www.youtube.com/playlist?list=PLoWGuY2dW-Acmc8V5NYSAXBxADMm1rE4p)) course lectured by Oscar Ramos for the Universidad de Ingeniería y Tecnología (UTEC), 2018.

The following study will be developed by the implementation of *Python* code for the application and visualization of the concepts learned with the use of *NumPy* and *Plotly* libraries. 

---

## Visual Setup

The plotting of the interactive graphic visuals will be done by a `Viewer` object based on the *Plotly*'s `Figure` object.

---

In [57]:
# Importing modules...
import numpy as np
import plotly.graph_objects as go

class Viewer: 
    def __init__(self, title, graphical=False, size=5):
        self.title = title
        self.graphical = graphical # Toggle to activate graphical mode
        self.size = size # Change graph dimensions 

        # Create Figure 
        self.figure = go.Figure(
            layout=go.Layout(
                height=700, 
                width=900,
                title=go.layout.Title(text=self.title)
            )
        )

        # Set up layout enviroment
        self.figure.update_layout(
            scene_aspectmode='cube',
            scene = dict(
                xaxis_title='x'*self.graphical,
                yaxis_title='y'*self.graphical, 
                zaxis_title='z'*self.graphical, 
                xaxis=dict(
                    range=[-self.size,self.size],
                    showbackground=self.graphical,
                    showticklabels=self.graphical,
                    showaxeslabels=self.graphical,
                    showgrid=self.graphical,
                    showspikes=self.graphical
                    ),
                yaxis=dict(
                    range=[-self.size,self.size],
                    showbackground=self.graphical,
                    showticklabels=self.graphical,
                    showaxeslabels=self.graphical,
                    showgrid=self.graphical,
                    showspikes=self.graphical
                    ), 
                zaxis=dict(
                    range=[-self.size,self.size],
                    showbackground=self.graphical,
                    showticklabels=self.graphical,
                    showaxeslabels=self.graphical,
                    showgrid=self.graphical,
                    showspikes=self.graphical
                    )
            )
        )

        # Change camera settings
        self.figure.update_layout(
            scene=dict(
                camera=dict(
                    projection=dict(
                        type='orthographic'
                    )
                )
            )
        )

    def add_frame(self, frame, name, color=None):

        # Set default colors
        axis_name_list = ['x', 'y', 'z']
        axis_color_list = ['red', 'green', 'blue']
        origin_color = 'black'

        # Color argument filled
        if color != None:
            axis_color_list = [color]*3 
            origin_color = color 

        self.figure.add_trace(
            go.Scatter3d(
                x=frame.t[0],
                y=frame.t[1],
                z=frame.t[2],
                mode='markers',
                marker=dict(
                    size=3,
                    opacity=0.80,
                    color=origin_color
                ),
                name='{'+name+'}',
                legendgroup='Frames',
                legendgrouptitle_text='Frames',
                showlegend=True
            )
        )

        for axis, axis_color in enumerate(axis_color_list):

            arrow = np.hstack((frame.t, frame.t + frame.R[:,axis].reshape(-1,1))) # Arrow of an axis

            self.figure.add_trace(
                go.Scatter3d(
                    x=arrow[0], 
                    y=arrow[1],
                    z=arrow[2], 
                    mode='lines',
                    line=dict(
                        width=3,
                        color=axis_color
                        ),
                    showlegend=False,
                    name=axis_name_list[axis]+name,
                    hoverinfo = None if self.graphical else 'skip'
                )
            )

    def add_point(self, point, name, color=None):
        self.figure.add_trace(
            go.Scatter3d(
                x=point[0],
                y=point[1],
                z=point[2],
                mode='markers',
                marker=dict(
                    size=3,
                    opacity=0.80,
                    color=color
                ),
                name=name,
                legendgroup='Points',
                legendgrouptitle_text='Points',
                showlegend=self.graphical
            )
        )

    def add_solid(self, vertices, name, color=None):
        self.figure.add_trace(
            go.Mesh3d(
                x=vertices[0],
                y=vertices[1],
                z=vertices[2],
                alphahull=0,
                color=color,
                flatshading=True,
                name=name,
                legendgroup='Objects',
                legendgrouptitle_text='Objects',
                showlegend=self.graphical
            )
        )

## Reference Frames

A Frame is essentially a way to stablish a spacial relation between elements. It can also be interpreted as the following:

- A **Coordinate System**;
- The base of a **Vector Space**;
- A representation of the **position** and **orientation** of a rigid body;

Important observations regarding frames:

- Completely defined by its **axes**;
- Its axes are usually unit vectors;
- Represented by the base of a vector space notation $\{F\}$;
- Elements represented in respect to a frame are superscripted by its name, e.g ${}^{F}E$.

---

In [58]:
class Frame: 
    def __init__(self, R=np.eye(3), t=np.zeros((3, 1))):
        self.R = R
        self.t = t

# Create graph viewer
viewer = Viewer('World Coordinate Frame', size=2)

# Plot main frame
viewer.add_frame(Frame(), 'W')

viewer.figure.show(renderer='notebook_connected') # Choose renderer: 'notebook' for offline mode and 'browser' for browser plots

## Generating Random Frames

The following code will help stablishing randomized values for easier and more varied visualization of the concepts. 

---

In [59]:
from scipy.spatial.transform import Rotation

# Random rotation matrix
def randR():
    rotation = np.eye(3)
    for axis in ['x', 'y', 'z']:
        rotation = Rotation.from_euler(axis,  np.random.randint(0, 360), degrees=True).as_matrix() @ rotation

    return rotation

# Random translation matrix
def randt(L): # Confined in a cube with an edge of length L
    return np.array([[2*L*np.random.random_sample()-L] for _ in range(3)])

# Create graph viewer
viewer = Viewer('World Coordinate Frame', size=5)

# Plot main frame
viewer.add_frame(Frame(), 'W')

# Generate and plot 3 random frames
for F in [['A', 'cyan'], ['B', 'magenta'], ['C', 'yellow']]:
    viewer.add_frame(Frame(randR(), randt(3)), *F);


viewer.figure.show(renderer='notebook_connected') # Choose renderer: 'notebook' for offline mode and 'browser' for browser plots

## Rotation Matrices

For each frame's perspective, their fundamental matrix that represents its axes will always be an identity matrix $I$ (or canonic). Therefore, to represent a frame in another frame's perspective, it is essential to know the spatial relationship between these frames. 

This spatial relationship is described by the **Rotation Matrix** between these frames. For now, the frames will have the same origin points.

The rotation matrices will be represented as: $$\begin{bmatrix} | & | & | \\ \hat{x} & \hat{y} & \hat{z} \\ | & | & | \end{bmatrix}$$

Interpreting the world in a frame's perspective is simple: assume that the known frame is represented by $I$. Seeing other frames in its perspective is just representing them by the rotation matrix that correlates the main frame in perspective to the observed frame. Mathematically:
1. Let a main frame $\{M\}$ be represented by $I$;
2. If an observed frame $\{O\}$ can be transformed into $\{M\}$ by a rotation matrix $R$, the frame $\{O\}$ will be represented by the rotation matrix ${}^{M}R_{O}$ in respect to the frame $\{M\}$;
3. Therefore, any given point ${}^{O}P$  can be viewed by the perspective of the frame $\{M\}$ by: $${}^{M}P = {}^{M}R_{O} \cdot {}^{O}P$$
4. The inverse action holds: $${}^{O}P = {}^{O}R_{M} \cdot {}^{M}P$$
5. Explicitly:
$${}^{M}R_{O}=\begin{bmatrix} \hat{x}_O \cdot \hat{x}_M & \hat{y}_O \cdot \hat{x}_M & \hat{z}_O \cdot \hat{x}_M  \\ \hat{x}_O \cdot \hat{y}_M & \hat{y}_O \cdot \hat{y}_M & \hat{z}_O \cdot \hat{y}_M  \\ \hat{x}_O \cdot \hat{z}_M & \hat{y}_O \cdot \hat{z}_M & \hat{z}_O \cdot \hat{z}_M \end{bmatrix}$$

---

In [60]:
# Rotate a point P in a observed frame O to be in respect to a main frame M 
def R(O, M):
    return np.array([[np.dot(O[0], M[0]), np.dot(O[1], M[0]), np.dot(O[2], M[0])],
                     [np.dot(O[0], M[1]), np.dot(O[1], M[1]), np.dot(O[2], M[1])],
                     [np.dot(O[0], M[2]), np.dot(O[1], M[2]), np.dot(O[2], M[2])]])


A, B = Frame(randR()), Frame(randR()) # Random frames in the origin
P = randt(1.5) # Random point in space

# Create graph viewer in {A}'s perspective
V_A = Viewer('Frame {A}\'s Perspective', graphical=True, size=2)

# Add frames
V_A.add_frame(Frame(), 'A', 'cyan')
V_A.add_frame(Frame(R(B.R, A.R)), 'B', 'magenta')
V_A.add_point(A.R @ P, 'P', 'black')

V_A.figure.show(renderer='notebook_connected') # Choose renderer: 'notebook' for offline mode and 'browser' for browser plots

# Create graph viewer in {B}'s perspective
V_B = Viewer('Frame {B}\'s Perspective', graphical=True, size=2)

# Add frames
V_B.add_frame(Frame(R(A.R, B.R)), 'A', 'cyan')
V_B.add_frame(Frame(), 'B', 'magenta')
V_B.add_point(B.R @ P, 'P', 'black')

V_B.figure.show(renderer='notebook_connected') # Choose renderer: 'notebook' for offline mode and 'browser' for browser plots


## Properties of Rotation Matrices

Rotation matrices follow these properties:
- ${}^{B}R_{A}={}^{A}R_{B}^{-1}={}^{A}R_{B}^{T}$
- ${}^{A}R_{Z}={}^{A}R_{B}\cdot {}^{B}R_{C} \cdot ... \cdot {}^{X}R_{Y}\cdot{}^{Y}R_{Z}$
- $\det(R) = +1$

It is possible to interpret rotation matrices as rotation operators for elements. Therefore, applying the same rotation matrix to all points in a point cloud will result it to be rotated as such. In this case:
- Post-multiplication will be applied when rotating in the **current** frame;
- Pre-multiplication will be applied when rotating in the **fixed** frame;

---

In [61]:
# Create graph viewer
viewer = Viewer('Rotation of N points around an Axis', graphical=True, size=2)

# Plot main frame
viewer.add_frame(Frame(), 'W')

# Point (1, 1, 1)
P = np.ones(3).reshape(-1, 1)

N = 6 # Number of points to visualize
axis = 'z' # Axis of rotation

# Plot P rotated around the chosen axis N times
for theta in range(N):
    viewer.add_point(Rotation.from_euler(axis, 360*theta//N, degrees=True).as_matrix() @ P, f'{360*theta//N}°', 'black');

viewer.figure.show(renderer='notebook_connected') # Choose renderer: 'notebook' for offline mode and 'browser' for browser plots

## Homogeneous Transformations

To represent a point in a rotated *and* translated observed frame $O$, the following relationship is used: $${}^{O}P={}^{O}R \cdot P + {}^{O}t$$

In matrix representation: $$\begin{bmatrix} {}^{O}P \\ 1 \end{bmatrix} = \begin{bmatrix} {}^{O}R & {}^{O}t \\ 0 & 1 \end{bmatrix} \cdot \begin{bmatrix} P \\ 1 \end{bmatrix}$$ 

And in a compact way: $${}^{O}\tilde{P} = {}^{O}H \cdot \tilde{P}$$

Where ${}^{O}H$ is the homogeneous transformation to reference frame $O$ and the tilded coordinates are homogeneous coordinates.

A **Homogeneous Transformation Matrix** $H$ represents both position and orientation (pose) of a frame in relation to another in a single matrix, such that:

$$H = \begin{bmatrix} r_{11} & r_{12} & r_{13} & t_{x} \\ r_{21} & r_{22} & r_{23} & t_{y} \\ r_{31} & r_{32} & r_{33} & t_{z} \\ 0 & 0 & 0 & 1 \end{bmatrix} = \begin{bmatrix} R & t \\ 0 & 1 \end{bmatrix}$$

For the inverse of a homogeneous transformation, the following formula applies: $$H^{-1}= \begin{bmatrix} R^T & -R^T t \\ 0 & 1 \end{bmatrix}$$

The main purpose of using homogeneous transformations is to linearize operations.

---

In [62]:
# Make coordinates homogeneous
def to_homo(points):
    return np.vstack((points, np.ones(points.shape[1]))) 

# Joins the rotation and translation matrices to make a homogeneous transformation
def join_homo(R, t):
    return np.vstack((np.hstack((R, t)), np.array([0, 0, 0, 1])))

# Extract the rotation and translation matrices from the homogeneous transformation
def split_homo(H):
    return H[0:3, 0:3], H[0:3 , [-1]]

# Returns the inverse of a homogeneous matrix
def inverse_homo(H):
    R, t = split_homo(H)
    
    return join_homo(R.T, -R.T @ t)

In [66]:
# Returns the vertices of an unit cube with corner in origin
def cube(position=np.zeros(3).reshape(-1,1), size=1/2):
    vertices = np.array([[(V>>2)&1, (V>>1)&1, (V>>0)&1] for V in range(8)]).T*size+position

    return vertices

# Create graph viewer
viewer = Viewer('Random Homogeneous Transformation of an Object', graphical=True, size=5)

# Plot main frame
viewer.add_frame(Frame(), 'W')

box = cube() # Creates a cube
box = to_homo(box)

# Generate random homogeneous transformation
randH = join_homo(randR(), randt(3)) 

# Rotate and translate in a homogeneous transformation
box = randH @ box 

box = box[:-1] # Extract regular coordinates

# Plot box
viewer.add_solid(box, name='Box', color='lightsteelblue')

viewer.figure.show(renderer='notebook_connected') # Choose renderer: 'notebook' for offline mode and 'browser' for browser plots